In [23]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

import math
import tensorflow as tf
import numpy as np
import scipy.io as sio
import scipy.sparse.linalg as linalg
import time
import warnings

# Filter out specific TensorFlow warnings
warnings.filterwarnings('ignore', message="Gradients do not exist for variables")
# Set TensorFlow logging to ERROR only
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging (1: INFO, 2: WARNING, 3: ERROR)
tf.get_logger().setLevel('ERROR')
# Ensure TensorFlow is using GPU if available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

# Use float64 for all operations
tf.keras.backend.set_floatx('float64')

# ================Parameters======================
r = 5        # underlying rank
n = 200      # size (num. of rows)
q = 200      # size (num. of columns)
m = 60       # size (number of measurements)
step_initial = 0.5    # initial value of step size (eta in the paper)
maxIt = 100    # num. of layers you want to train
thr_initial = 0.7

# =============Generate Data y_k = A_k U b_k=============
def generate_problem(r, n, q):
    U0_t = tf.random.normal((n, r), dtype=tf.float64) / math.sqrt(n)
    U0_t, _ = tf.linalg.qr(U0_t)
    B0_t = tf.random.normal((r, q), dtype=tf.float64) / math.sqrt(q)
    A = tf.random.normal((m, n, q), dtype=tf.float64) / math.sqrt(m)
    noise = tf.random.normal((n, q), dtype=tf.float64) / math.sqrt(n * q)
    Y0_t_array = tf.TensorArray(dtype=tf.float64, size=q)
    for k in range(q):
        yk= tf.matmul(A[:, :, k], tf.matmul(U0_t, B0_t[:, k][:, tf.newaxis]))[:, 0]
        Y0_t_array = Y0_t_array.write(k, yk)
    # Convert the TensorArray to a Tensor
    Y0_t = Y0_t_array.stack()
    Y0_t = tf.transpose(Y0_t)  # Make sure the shape matches the expected (m, q)

    return U0_t, B0_t, Y0_t, A


class MatNet(tf.keras.Model):
    def __init__(self, step_initial, thr_initial):
        super(MatNet, self).__init__()
        #self.maxIt = maxIt
        self.step_initial = step_initial
        self.thr_initial = thr_initial

        # Define the parameters to be learned
        self.step = [self.add_weight(name=f'step_{t}',
                                     shape=(),
                                     initializer=tf.constant_initializer(step_initial),
                                     dtype=tf.float64,
                                     trainable=True) for t in range(maxIt)]

        self.thr = [self.add_weight(name='thr',
                                    shape=(),
                                    initializer=tf.constant_initializer(thr_initial),
                                    dtype=tf.float64,
                                    trainable=True)]
    def lowrank(self, X0, threshold):
        X0 = tf.identity(X0)  # Clone the tensor
        St, Ut, Vt = tf.linalg.svd(X0, compute_uv=True)
        #print(threshold)
        thres = threshold * St[0]
        #print(f"thres: {thres}")
        St = tf.maximum(St - thres, 0)  # Equivalent to relu(St - thres)
        St = tf.cast(St, tf.float64)  # Ensure St is of float type
        r = tf.math.count_nonzero(St)
        #print(f"r: {r}")
        #print(f"St: {St}")
        S_diag = tf.linalg.diag(St)
        Xinit = Ut @ S_diag @ tf.transpose(Vt)
        # Since TensorFlow does not have svd_lowrank, we recompute SVD for Xinit as a demonstration.
        # This might not be exactly what you want, so adjust according to your specific needs.
        St, Ut, Vt = tf.linalg.svd(Xinit, compute_uv=True)
        return tf.cast(Ut[:,0:r], tf.float64)  # Cast Ut to double
    def call(self, Y0_t, U0_t, B0_t, A, num_l):
        # Initialization
        n, r0 = U0_t.shape
        #print(f"r0: {r0}")
        r0, q = B0_t.shape
        # Initialize X0 as a list of tensors to be concatenated later
        X0_list = []
        for k in range(q):
            X0_k = tf.matmul(tf.transpose(A[:, :, k]), Y0_t[:, k][:, tf.newaxis])
            X0_list.append(X0_k[:, 0])  # Remove the extra dimension
        X0 = tf.stack(X0_list, axis=1)
        U_t = self.lowrank(X0, self.thr[0])
        n, r = U_t.shape
        #print(r)
        B_t_list = []
        for k in range(q):
            B_t_k = tf.matmul(tf.linalg.pinv(tf.matmul(A[:, :, k], U_t)), Y0_t[:, k][:, tf.newaxis])  # Column-wise Least Squares
            B_t_list.append(B_t_k[:, 0])
        B_t = tf.stack(B_t_list, axis=1)
        # Main Loop
        for t in range(1, num_l):
            E_t = tf.zeros((n, r), dtype=tf.float64)
            for k in range(q):
                AkU = tf.matmul(A[:, :, k], U_t)
                Yk_minus_AkU_Bk = Y0_t[:, k] - tf.matmul(AkU, B_t[:, k][:, tf.newaxis])[:, 0]
                E_t += tf.matmul(A[:, :, k], Yk_minus_AkU_Bk[:, tf.newaxis], transpose_a=True) * B_t[:, k]

            #print(f"Gradient: {E_t}")
            Unew = tf.linalg.qr(U_t + self.step[t] * E_t)[0]  # Projected Gradient Descent
            Bnew_list = []
            for k in range(q):
                Bnew_k = tf.matmul(tf.linalg.pinv(tf.matmul(A[:, :, k], Unew)), Y0_t[:, k][:, tf.newaxis])  # Column-wise Least Squares
                Bnew_list.append(Bnew_k[:, 0])
            Bnew = tf.stack(Bnew_list, axis=1)
            U_t, B_t = Unew, Bnew

        Y_t_list = []
        for k in range(q):
            Y_t_k = tf.matmul(A[:, :, k], tf.matmul(U_t, B_t[:, k][:, tf.newaxis]))[:, 0]
            Y_t_list.append(Y_t_k)
        Y_t = tf.stack(Y_t_list, axis=1)
        loss = tf.norm(Y_t - Y0_t)
        #print(loss)
        return loss, U_t, B_t
    def enable_single_layer(self, layer_index):
    # Assuming 'model' is a tf.keras.Sequential model or a model that supports indexing.
      for i, layer in enumerate(self.layers):
        layer.trainable = (i == layer_index)
    def enable_layers(self, num_layers):
    # Enable first 'num_layers' and disable the rest.
      for i, layer in enumerate(self.layers):
        layer.trainable = i < num_layers


Num GPUs Available:  0


In [24]:
import tensorflow as tf
from tensorflow import keras
import scipy.io as sio
import numpy as np
import time

Nepoches_pre = 4
Nepoches_full = 4
lr_fac = 1.0  # basic learning rate
maxIt = 10 # define the maximum iterations based on your model's depth

net = MatNet(step_initial, thr_initial)
start = time.time()

for stage in range(1, maxIt):
    print(f'Layer {stage}, Pre-training ======================')
    if stage > 6:
        Nepoches_full = 2

    net.enable_single_layer( stage)
    # Define a separate optimizer for this stage
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_fac * 0.01 if stage == 0 else lr_fac * 0.1)
    for epoch in range(Nepoches_pre):
        U0_t, B0_t, Y0_t, A = generate_problem(r, n, q)
        with tf.GradientTape() as tape:
            loss, U_t, B_t = net(Y0_t, U0_t, B0_t, A, stage + 1)  # Adjust based on your model's method signature
            gradients = tape.gradient(loss, net.trainable_variables)
            optimizer.apply_gradients(zip(gradients, net.trainable_variables))

        if epoch % 2 == 0:
            print(f"epoch: {epoch} \t loss: {loss.numpy()}")

    print(f'Layer {stage}, Full-training =====================')
    net.enable_layers(stage + 1)
    for epoch in range(Nepoches_full):
        U0_t, B0_t, Y0_t, A = generate_problem(r, n, q)
        with tf.GradientTape() as tape:
            loss, U_t, B_t = net(Y0_t, U0_t, B0_t, A, stage + 1)  # Adjust based on your model's method signature
            gradients = tape.gradient(loss, net.trainable_variables)
            optimizer.apply_gradients(zip(gradients, net.trainable_variables))

        if epoch % 2 == 0:
            print(f"epoch: {epoch} \t loss: {loss.numpy()}")

end = time.time()
print(f"Training end. Time: {end - start}")

Layer 1, Pre-training ======================
epoch: 0 	 loss: 0.462218318014601
epoch: 2 	 loss: 0.38696277976393384
Layer 1, Full-training =====================
epoch: 0 	 loss: 0.27695845083330706
epoch: 2 	 loss: 0.2793890824935758
Layer 2, Pre-training ======================
epoch: 0 	 loss: 0.13551270823753866
epoch: 2 	 loss: 0.12119772680084283
Layer 2, Full-training =====================
epoch: 0 	 loss: 0.12256420289648025
epoch: 2 	 loss: 0.11994938553310999
Layer 3, Pre-training ======================
epoch: 0 	 loss: 0.06425096751506156
epoch: 2 	 loss: 0.05858813710029684
Layer 3, Full-training =====================
epoch: 0 	 loss: 0.048606123434752596
epoch: 2 	 loss: 0.05587164654596481
Layer 4, Pre-training ======================
epoch: 0 	 loss: 0.03552836213153395
epoch: 2 	 loss: 0.027162113972454525
Layer 4, Full-training =====================
epoch: 0 	 loss: 0.026853590439603024
epoch: 2 	 loss: 0.0237681482546749
Layer 5, Pre-training ======================
epoc

In [25]:
net.trainable_weights

[<tf.Variable 'step_0:0' shape=() dtype=float64, numpy=0.5>,
 <tf.Variable 'step_1:0' shape=() dtype=float64, numpy=3.3549546879366843>,
 <tf.Variable 'step_2:0' shape=() dtype=float64, numpy=0.5292875623123038>,
 <tf.Variable 'step_3:0' shape=() dtype=float64, numpy=1.7927214018335142>,
 <tf.Variable 'step_4:0' shape=() dtype=float64, numpy=1.1939650406274556>,
 <tf.Variable 'step_5:0' shape=() dtype=float64, numpy=0.603998130128475>,
 <tf.Variable 'step_6:0' shape=() dtype=float64, numpy=0.6594405072753532>,
 <tf.Variable 'step_7:0' shape=() dtype=float64, numpy=0.830699251849397>,
 <tf.Variable 'step_8:0' shape=() dtype=float64, numpy=0.5604643375089539>,
 <tf.Variable 'step_9:0' shape=() dtype=float64, numpy=0.5262913880991335>,
 <tf.Variable 'thr:0' shape=() dtype=float64, numpy=0.7000000000162544>]